In [1]:
import re
import pandas as pd
from tabulate import tabulate
from fetch_process_function import fetch_and_save_erc20_transfers, process_transactions, calculate_pnl, process_and_save_address
'''
data_process.ipynb和data_fetch.ipynb同样功能，但是通过封装函数实现了给日期地址等参数就可以返回数据，更加简洁
添加了pnl的计算，我们这里只计算第一种交易的pnl，其余的暂时忽略不计。
最后对于address也进行一个处理
'''
# ADDRESS = "0x5be9a4959308A0D0c7bC0870E319314d8D957dBB"
# ADDRESS = "0x2c89a2ee92b9870f55989b4132a58c0e85222d86"
ADDRESS = "0x6c2a355929ee1262305e385ad49b84fe5f5a4777"

START_DATE = "2024-12-01 08:00" # Singapore Time，the function will convert it to UTC
END_DATE = "2025-01-01 08:00"
OUTPUT_FILE = "C:/Users/YuweiCao/Documents/GitHub/Project/Project/etherscan/result"
API_KEY = "VQAIR728IM4Z8RZKPYBR4ESM5I3WBZK2C1"
BASE_TOKENS = {"USDT", "USDC", "USDE", "DAI"}
base_pattern = re.compile('|'.join(BASE_TOKENS))

In [2]:

transfers_df = fetch_and_save_erc20_transfers(
        address=ADDRESS,
        start_date=START_DATE,
        end_date=END_DATE,
        output_file=OUTPUT_FILE,
        offset=1000,
    )
transfers_df = pd.read_csv(OUTPUT_FILE + "/erc20_transfers.csv")
record_df_1, record_df_2, record_df_3 = process_transactions(transfers_df, OUTPUT_FILE, ADDRESS, base_pattern)
# 这里我为了显示简洁把process_transactions中print结果的注释掉了 如果需要可以取消注释
if not record_df_1.empty:
    unique_tokens_count = record_df_1["token_symbol"].nunique()
    unique_tokens = record_df_1["token_symbol"].unique()
    results = []
    # 在USDC USDE USDT DAI的价格默认为1：1的情况下，不考虑不同的稳定币作为单位的情况

    for token in unique_tokens:
        token_df = record_df_1[record_df_1["formatted_record"].str.contains(fr"\b{token}\b", regex=True)]
        unique_price_units = token_df["price_unit"].unique()
        price_units_str = ", ".join(unique_price_units)
        
        print(f"{token} (Price Units: {price_units_str}): {len(token_df)} records")

        pnl_df = calculate_pnl(token_df)
        print(tabulate(pnl_df, headers='keys', tablefmt='grid', showindex=False, floatfmt=".2f"))   
        total_pnl = pnl_df["pnl"].sum()

        results.append({"token": token, "price_unit": price_units_str, "total_pnl": total_pnl})

    pnl_summary_df = pd.DataFrame(results)

    print(tabulate(pnl_summary_df, headers='keys', tablefmt='grid', showindex=False, floatfmt=".2f"))
else:
    print("No transactions found")


2025-01-15 15:39:40,423 - INFO - Start block: 21303933, End block: 21525891
2025-01-15 15:39:40,423 - INFO - Fetching ERC20 transfers from block 21303933 to 21525891...
2025-01-15 15:39:42,040 - INFO - Page 1: Retrieved 1000 transactions.
2025-01-15 15:39:43,428 - INFO - Page 2: Retrieved 1000 transactions.
2025-01-15 15:39:44,809 - INFO - Page 3: Retrieved 1000 transactions.
2025-01-15 15:39:46,411 - INFO - Page 4: Retrieved 1000 transactions.
2025-01-15 15:39:48,245 - INFO - Page 5: Retrieved 1000 transactions.
2025-01-15 15:39:49,647 - INFO - Page 6: Retrieved 1000 transactions.
2025-01-15 15:39:51,014 - INFO - Page 7: Retrieved 1000 transactions.
2025-01-15 15:39:52,115 - INFO - Page 8: Retrieved 208 transactions.
2025-01-15 15:39:52,999 - WARNING - API request failed on page 9: Request failed: No transactions found
2025-01-15 15:39:52,999 - INFO - Final dataset contains 7208 unique transactions.
2025-01-15 15:39:53,099 - INFO - Data successfully saved to C:/Users/YuweiCao/Document

...Merging 1941 groups of transactions...
Transaction Hash 0x00c5efc959654417d5ace350856ee4e0ccbfbb8cb0c307dadb5d0c389e968e02 contains 4 records:
Transaction Hash 0x00cd2a41678a5b7aae4c1e97a80b4865411c19965235d5500e2ebc9c3238acf9 contains 4 records:
Transaction Hash 0x00e1b40f23ffc2d0967a96cac8a1cb4de92e7ab4e7f2617eabd8a512db2f4945 contains 10 records:
Transaction Hash 0x011169c6efee55b084b0d71443ec65b0d728def27ac5aae803f2d6d07ab16133 contains 4 records:
Transaction Hash 0x01f5554e71c97b1a51b43b64ccceeaef2039e1b034f75a0bb2d3f01dc68dbd3f contains 8 records:
Transaction Hash 0x05afc7463a71669cd8fffe7cc05cd69e7a5ec77e0e5a276cbcdeffac84a2f8eb contains 8 records:
Transaction Hash 0x0611d77a4857c12e708edf1d7dcc164b21282f5c49fb0110042d2e8af4932401 contains 6 records:
Transaction Hash 0x06315896fd8b9ed4f5235cacd7437fe02c9adfaf6a9b0fd43d1a4ea9d1f7c84b contains 4 records:
Transaction Hash 0x0653b419416141bf0448a003d3db127d9cd2977c07f413acd31a556b9a19d63a contains 4 records:
Transaction Hash 0x07

In [ ]:
TOP_ACCOUNTS_DF = pd.read_csv(OUTPUT_FILE + "/top_accounts.csv")
transfers_df = pd.read_csv(OUTPUT_FILE + "/erc20_transfers.csv")
output_file = OUTPUT_FILE + "/address_counts_with_name_tags.csv"

process_and_save_address(transfers_df, TOP_ACCOUNTS_DF, output_file)

In [ ]:
address_counts = pd.read_csv(OUTPUT_FILE + "/address_counts_with_name_tags.csv")
address_threshold = 10
important_addresses = address_counts[address_counts["total_value_count"] > address_threshold]
important_addresses = important_addresses.sort_values(by="total_value_count", ascending=False)
# important_addresses = important_addresses[important_addresses["address"] != ADDRESS.lower()]

print(important_addresses)
